In [ ]:
import boto3
import io
import numpy as np
import json
import random
random.seed()

In [ ]:
ACT_LABELS = ["dws","ups", "wlk", "jog", "std", "sit"]

In [ ]:
def load_data(bucket_name,key):
    s3_client = boto3.client('s3')
    obj = s3_client.get_object(Bucket=bucket_name, Key=key)
    
    with io.BytesIO(obj["Body"].read()) as f:        
        f.seek(0)
        data = np.load(f)
    return data

In [ ]:
bucket_name = 'har-data-105064114710' # name of the bucket

testX = load_data(bucket_name,'test/test_data.npy')
testy = load_data(bucket_name,'test/test_labels.npy')

In [ ]:
print (testX.shape,testy.shape)

n_timesteps, n_features, n_outputs = testX.shape[2], testX.shape[1], testy.shape[1]
print(n_timesteps, n_features, n_outputs)
n_steps, n_length = 4, 32
#n_steps, n_length = 1, 128
input_data = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
 
print('reshape=',input_data.shape)

In [ ]:
r = random.randint(0,input_data.shape[0])
print(r)
print(input_data[r].shape)

inference_input=input_data[r:r+1]
print('inference_input=',inference_input.shape)

ep_name = 'ep-har-tf-2021-05-24-02-37-04'

client = boto3.client('sagemaker-runtime')

payload=inference_input.tolist()

response = client.invoke_endpoint(EndpointName=ep_name,
                                   ContentType='application/json',
                                   Body=json.dumps(payload)
                                 )

result = json.loads(response['Body'].read().decode())

predictions=result['predictions'][0]

print('r=',r)
for i in range(0,len(predictions)):

    print('class=',ACT_LABELS[i], 'proba=',round(float(predictions[i]),4), 'truth=',testy[r][i])


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

x = testX[r]

df = pd.DataFrame(x[0])

for i in range(1,12):
    df.insert(i, i, x[i], True)
    
df.columns=(['attitude.roll','attitude.pitch','attitude.yaw','gravity.x','gravity.y','gravity.z','rotationRate.x','rotationRate.y','rotationRate.z','userAcceleration.x','userAcceleration.y','userAcceleration.z'])

df

In [ ]:
df.plot(subplots=True,figsize = (20,30))
#plt.tight_layout()
plt.show()